# Sainfoin Seed Classification FasterRCNN Model

### For Colab Users
Run the code cell below to mount your Google Drive. Change the `BASE_DIR` variable to the folder location of the cloned GitHub folder relative to your Google Drive, so something along the lines of 'drive/MyDrive/path/to/dir/repo_folder'.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = 'drive/MyDrive/github_repos/sainfoin_seed_RCNN'
%cd -q $BASE_DIR


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import datetime
str(datetime.date.today())

'2023-09-03'

In [ ]:
!pip install -r colab_requirements.txt

### Module Import

In [ ]:
import os
import sys
import cv2
import torch
import torchvision
import torchmetrics
import yaml
import time
import logging

import pandas as pd
import numpy as np
import albumentations as A

from glob import glob
from tqdm import tqdm
from random import randint
from albumentations.pytorch import ToTensorV2
from torchmetrics.classification import MulticlassAccuracy, MulticlassJaccardIndex
from torchmetrics.detection import IntersectionOverUnion, MeanAveragePrecision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import ToTensor
from torchvision.ops import nms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

### Import Custom Code and Configuration Settings

In [ ]:
sys.path.append('./src')
from config import parse_config
from loggers import create_logger
from model import create_model
from transforms import train_transforms, val_transforms, collate_fn
from datasets import SeedDataset, dir_sampler
from data_splitting import train, val, test
from train_val import train_model
from config import train_dir, val_dir, test_dir, annotation_dir, chkpt_dir, tensorboard_dir, log_dir
from config import device, cores, classes, n_classes, resize_to, n_epochs, batch_size
from config import base_name, lr, momentum, gamma



# train_dir, val_dir, test_dir, annot_dir,\
# chkpt_dir, tb_dir, log_dir, device, cores,\
# classes, n_classes, resize_to, n_epochs, batch_size,\
# base_name, lr, momentum, gamma = parse_config('config.yml')

### Set up Loggers
This cell is meant to be set up only once! If you run it multiple times, you will end up with many duplicates of each logging message.

In [ ]:
logger = create_logger()
logger.info('Training notebook started')

2023-09-03 17:42:31,400:main_app:INFO: - Training notebook started


### Log System Information

In [ ]:
os_details = !lsb_release -a
os_details = '\n'.join(os_details)

cpu_info = !lscpu
cpu_info = '\n'.join(cpu_info)

sys_info = 'Python ' + sys.version

headings = ['OS_DETAILS:\n', 'CPU_INFO:\n', 'PYTHON KERNEL:\n']
details = [os_details, cpu_info, sys_info]

for h, d in zip(headings, details):
  logger.info(h + d)

2023-09-03 17:42:31,633:main_app:INFO: - OS_DETAILS:
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy
2023-09-03 17:42:31,635:main_app:INFO: - CPU_INFO:
Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  12
  On-line CPU(s) list:   0-11
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               85
    Thread(s) per core:  2
    Core(s) per socket:  6
    Socket(s):           1
    Stepping:            7
    BogoMIPS:            4400.30
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscal
                         l nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopo
                   

In [ ]:
# Validate torch device settings
if device=='cuda':
  try:
    assert torch.cuda.is_available(), 'No CUDA device is available.'
    logger.info(f"CUDA device name: {torch.cuda.get_device_name()}")
    logger.info(f"CUDA device capabilities: {torch.cuda.get_device_capability()}")
    logger.info(f"CUDA device properties: {torch.cuda.get_device_properties(0)}")
  except AssertionError as e:
    logger.warning(e, exc_info=True)
    logger.info("Overwriting model config and setting device to 'cpu'.")
    device = torch.device('cpu')


2023-09-03 17:42:31,708:main_app:INFO: - CUDA device name: NVIDIA A100-SXM4-40GB
2023-09-03 17:42:31,710:main_app:INFO: - CUDA device capabilities: (8, 0)
2023-09-03 17:42:31,711:main_app:INFO: - CUDA device properties: _CudaDeviceProperties(name='NVIDIA A100-SXM4-40GB', major=8, minor=0, total_memory=40513MB, multi_processor_count=108)


### Model Training and Evaluation
We will build individual models for each of a range of training sample sizes. The total image dataset is 500 images, thus an 80/20 split on the data results in 400 training images and 100 validation images. The training sizes will be split up as follows:
```
[20, 40, 80, 200, 400]
```
with corresponding validation sizes of
```
[5, 10, 20, 50, 100]
```
So 5%, 10%, 20%, 50%, and 100% of the total available data for each group

In [ ]:
# Set proportion sizes for subsampling the data
# sizes = [0.05, 0.1, 0.2, 0.5, 1.0]
sizes = [0.05, 0.5, 1.0]

# Read in all of the image metadata
img_data = pd.read_csv('./data/power_analysis/results.csv')
print(img_data.head())


   Unnamed: 0 img_id   img_name  class                variety     method  \
0           0  12600  12600.jpg  train  Rocky Mountain Remont  HLDP7x35s   
1           1  9adc6  9adc6.JPG  train               Shoshone       BT3X   
2           2  1cd49  1cd49.jpg  train       AAC Mountainview  HLDP7x35s   
3           3  6eb3d  6eb3d.JPG  train               Shoshone       BT3X   
4           4  7b24d  7b24d.JPG  train               Shoshone       BT3X   

   sample_mass_g  rep  legume_fruit_mass_g  legume_seed_mass_g comments  
0              2    7               2.0189               1.569      NaN  
1              2    5               2.0131               1.677      NaN  
2              4    1               4.0014               2.921      NaN  
3              1   10               1.0075               0.887      NaN  
4              3    8               3.0066               2.736      NaN  


In [ ]:
img_data.loc[img_data['class']=='train'].shape

(400, 11)

In [11]:
# Start Tensorboard writer
model_config = [
    "model_name:\tfasterrcnn",
    f"pretrained:\tTrue",
    f"classes:\t{classes}",
    f"n_classes:\t{n_classes}",
    f"lr:\t{lr}",
    f"momentum:\t{momentum}",
    f"n_epochs:\t{n_epochs}",
    f"batch_size:\t{batch_size}",
    f"lr_scheduler_gamma:\t{gamma}"
]
logger.info("MODEL_CONFIG\n"+"\n".join(model_config))

train_imgs = img_data.loc[img_data['class']=='train']
val_imgs = img_data.loc[img_data['class']=='val']

logger.info("Starting training scenarios.")
for size in sizes:
  today = str(datetime.date.today())
  writer = SummaryWriter(os.path.join(tensorboard_dir, f"{base_name}_{size}_{today}"))
  logger.info("Starting Tensorboard Summary Writer.")
  logger.info(f"Scenario {sizes.index(size)+1}: training on imageset of {img_data.shape[0]*size}")
  logger.info(f"Creating train/validation splits for {size*100}% of the imageset in an 80/20 train/val split")
  if size < 1:
    train, _ = train_test_split(train_imgs,
                                test_size=size,
                                train_size=size,
                                stratify=train_imgs[['method']])
    _, val = train_test_split(val_imgs,
                              test_size=size,
                              train_size=size,
                              stratify=val_imgs[['method']])
  else:
    train = train_imgs.copy()
    val = val_imgs.copy()

  logger.info(f"Sampled images.\nTraining set: {train.shape[0]} images.\nVal set: {val.shape[0]} images")

  try:
    model_name = base_name + f"_{size}"
    model = create_model(n_classes, n_obj_det=500)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
  except:
    logger.exception('An exception occurred.')

  # Create train dataset
  train_dataset = SeedDataset(image_dir=train_dir,
                              annot_dir=annotation_dir,
                              resize_dims=(resize_to, resize_to),
                              classes=classes,
                              transforms=train_transforms(),
                              subset=list(train['img_name'].unique()))

  # Create val dataset
  val_dataset = SeedDataset(image_dir=val_dir,
                            annot_dir=annotation_dir,
                            resize_dims=(resize_to, resize_to),
                            classes=classes,
                            transforms=train_transforms(),
                            subset=list(val['img_name'].unique()))

  train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn,
  )

  val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
  )

  train_model(model=model,
              optimizer=optimizer,
              scheduler=scheduler,
              n_epochs=n_epochs,
              device=device,
              train_loader=train_loader,
              val_loader=val_loader,
              logger=logger,
              writer=writer,
              model_name=model_name)

  writer.close()

2023-09-03 17:42:31,747:main_app:INFO: - MODEL_CONFIG
model_name:	fasterrcnn
pretrained:	True
classes:	{'0': 'background', '1': 'split', '2': 'seed', '3': 'pod'}
n_classes:	4
lr:	0.01
momentum:	0.8
n_epochs:	100
batch_size:	4
lr_scheduler_gamma:	0.9
2023-09-03 17:42:31,750:main_app:INFO: - Starting training scenarios.
2023-09-03 17:42:31,760:main_app:INFO: - Starting Tensorboard Summary Writer.
2023-09-03 17:42:31,761:main_app:INFO: - Scenario 1: training on imageset of 25.0
2023-09-03 17:42:31,763:main_app:INFO: - Creating train/validation splits for 5.0% of the imageset in an 80/20 train/val split
2023-09-03 17:42:31,771:main_app:INFO: - Sampled images.
Training set: 20 images.
Val set: 5 images
2023-09-03 17:42:35,269:main_app:INFO: - Initializing training sequence
2023-09-03 17:42:35,270:main_app:INFO: - Epoch: 1
LR: [0.01]
Val Loss: 1.3026: 100%|██████████| 2/2 [00:08<00:00,  4.24s/it]
2023-09-03 17:43:16,475:main_app:INFO: - Epoch: 2
LR: [0.009000000000000001]
Val Loss: 1.2317: 1